In [10]:
#!pip3 install uvicorn fastapi pyngrok


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# 서버 관리용 fastapi 의존 라이브러리
import uvicorn

# fastapi 라이브러리
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles
from fastapi import File, UploadFile
from fastapi.responses import JSONResponse, HTMLResponse


# 인터페이스 데이터 관리를 위한 라이브러리
from pydantic import BaseModel

# 모델 Load 용 라이브러리
import PCB_Product_Model
import PCB_Defect_Model

# 데이터 분석용 라이브러리
import pandas as pd
import numpy as np
import os
import cv2
from ultralytics import YOLO

# ngrok을 활용하여 분석서버를 Global하게 띄우기 위한 라이브러리
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [7]:
# CORS issue 때문에 미들웨어 추가

from fastapi.middleware.cors import CORSMiddleware
origins = ["*"]

app = FastAPI(title="ML API")

# CORS 미들웨어 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 origin 허용
    allow_credentials=True,
    allow_methods=["GET", "POST", "PUT", "DELETE"],
    allow_headers=["*"],
)

### 1. 모델 불러오기

In [2]:
# 제품 분류 모델 (CNN)

In [3]:
model_path = 'pcb_prod_model_0708.h5'
class_path = 'pcb_class_dict_0708.json'

pcb_prod_model = PCB_Product_Model.Model_load(model_path, class_path)


In [4]:
# 제품 양불 판정 모델 (YOLO)

In [5]:
model_pt_path = 'best.pt'
pcb_defect_model = PCB_Defect_Model.Model_load(model_pt_path)

### 2. 예측 시뮬레이션

In [8]:
@app.get('/')
async def load_main():
    return FileResponse('index.html')
    

@app.post("/predict")
async def upload_image(file: UploadFile = File(...)):

    contents = await file.read()

    # 이미지 읽기
    nparr = np.frombuffer(contents, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    prod_type = pcb_prod_model.predict_image(False, img)

    prod_defect_dict = pcb_defect_model.predict_defect(img)

    error_type_list = prod_defect_dict['defect_name_list']
    defect_box_list = prod_defect_dict['image']
    
    # img_str = ndarray_to_base64(defect_image)
    
    result = {
        "product_type": prod_type,
        "error_type_list": error_type_list,
        "image": defect_box_list
    }

    return result


### 3.

In [ ]:
# colab 서버 주소는 실행 될 때마다 바뀌기 때문에 ngrok이라는 것을 활용하여서 고정적인 url로 들어올 수 있는 서버를 만들 수 있다. 아래의 코드를 활용하면 된다.

import nest_asyncio
from pyngrok import ngrok
import uvicorn

AUTH_TOKEN = "2hRkhpGbfhVbsBcpYewFZne3I4z_24P9h5EchPjzVTMJe6zEL"
ngrok.set_auth_token(AUTH_TOKEN)
ngrokTunnel = ngrok.connect(8000)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)



공용 URL https://c1e1-118-34-210-68.ngrok-free.app


INFO:     Started server process [22204]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     211.231.103.92:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     211.231.103.107:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
1/1 [==============================] - 0s 203ms/step

0: 352x640 1 short, 212.8ms
Speed: 6.4ms preprocess, 212.8ms inference, 3.5ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs\detect\predict7

0: 352x640 1 short, 181.7ms
Speed: 4.4ms preprocess, 181.7ms inference, 1.5ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 short, 165.1ms
Speed: 4.3ms preprocess, 165.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)
INFO:     118.34.210.65:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 64ms/step

0: 352x640 1 short, 187.3ms
Speed: 5.0ms preprocess, 187.3ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs\detect\predict7

0: 352x640 1 short, 194.9ms
Speed: 5.3ms preprocess, 194.9ms inference, 1.0ms postprocess per ima

t=2024-07-16T14:00:12+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=942087a1ae4b clientid=1d140dc6d467b9bb619f6d135b657457
t=2024-07-16T14:00:12+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=b55210c961a2 err="read tcp 172.16.248.191:56127->3.133.228.214:443: wsarecv: An established connection was aborted by the software in your host machine."
t=2024-07-16T14:00:12+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2024-07-16T14:00:13+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2024-07-16T14:00:14+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="faile

1/1 [==============================] - 0s 85ms/step

0: 352x640 1 short, 222.0ms
Speed: 6.6ms preprocess, 222.0ms inference, 8.6ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs\detect\predict7

0: 352x640 1 short, 169.8ms
Speed: 4.7ms preprocess, 169.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 short, 159.0ms
Speed: 3.0ms preprocess, 159.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 640)
INFO:     118.34.210.65:0 - "POST /predict HTTP/1.1" 200 OK
1/1 [==============================] - 0s 73ms/step

0: 352x640 1 short, 178.9ms
Speed: 3.5ms preprocess, 178.9ms inference, 2.1ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs\detect\predict7

0: 352x640 1 short, 199.2ms
Speed: 5.7ms preprocess, 199.2ms inference, 1.4ms postprocess per image at shape (1, 3, 352, 640)

0: 352x640 1 short, 176.8ms
Speed: 3.0ms preprocess, 176.8ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 640

t=2024-07-16T17:37:09+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=f2cc7e258f94 clientid=1d140dc6d467b9bb619f6d135b657457
t=2024-07-16T17:37:09+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=b55210c961a2 err="session closed"
t=2024-07-16T18:21:05+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=4fd398be2007 clientid=1d140dc6d467b9bb619f6d135b657457
t=2024-07-16T18:21:05+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=b55210c961a2 err="read tcp 172.16.248.191:62029->3.134.73.173:443: wsarecv: An existing connection was forcibly closed by the remote host."
t=2024-07-16T18:21:13+0900 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="failed to dial ngrok server with address \"connect.us.ngrok-agent.com:443\": dial tcp: lookup connect.us.ngrok-agent.com: no such host"
t=2024-07-16T19:23:25+0900 

INFO:     27.0.238.68:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed
INFO:     27.0.238.70:0 - "GET /predict HTTP/1.1" 405 Method Not Allowed


t=2024-07-17T02:40:08+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=5ada9e891273 clientid=1d140dc6d467b9bb619f6d135b657457
t=2024-07-17T02:40:08+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=b55210c961a2 err="session closed"
t=2024-07-17T05:02:19+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=bfd390ea68fa clientid=1d140dc6d467b9bb619f6d135b657457
t=2024-07-17T05:02:19+0900 lvl=eror msg="session closed, starting reconnect loop" obj=tunnels.session obj=csess id=b55210c961a2 err="session closed"
t=2024-07-17T09:24:26+0900 lvl=eror msg="heartbeat timeout, terminating session" obj=tunnels.session obj=csess id=78af368a2fb6 clientid=1d140dc6d467b9bb619f6d135b657457
t=2024-07-17T09:24:26+0900 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": dial tcp: lookup update.equinox.io: no such host"
t=2024-07-17T09:2